Coding up mass-radius-eccentricty hierarhcial Bayesian model in PyStan.

In [14]:
from __future__ import division, print_function
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["font.size"] = 20
import os
import requests
import pandas as pd
from cStringIO import StringIO
import numpy as np
import pystan


# load data from input files
# TTV amplitudes
inputdata1 = "../data/input/TTVamplitudes.twos.t2.ready.for.jags.3_NAamps_posreal_uncs2.0.txt"
fx1 = open(inputdata1,'r')
datax1 = fx1.read()    
# Get column names
firstlinex1 = datax1.split('\n', 1)[0].split(' ')
firstlinex1 = [w.replace('"', '') for w in firstlinex1]
firstlinex1 = [w.replace('.', '_') for w in firstlinex1]
#print firstlinex1
fx1.close()
# organize data from each file name in list
datax1 = np.genfromtxt(inputdata1, dtype=None, skip_header=1,names=firstlinex1 )

# Outer planet pair data
inputdata2 = "../data/input/Planet_and_Star_data_from_Q1_to_Q12_period_sorting_Pouter_2_sorted.txt"
fx2 = open(inputdata2,'r')
datax2 = fx2.read()    
# Get column names
firstlinex2 = datax2.split('\n', 1)[0].split(' ')
firstlinex2 = [w.replace('"', '') for w in firstlinex2]
firstlinex2 = [w.replace('.', '_') for w in firstlinex2]
#print firstlinex1
fx2.close()
# organize data from each file name in list
datax2 = np.genfromtxt(inputdata2, dtype=None, skip_header=2,names=firstlinex2 )

# Inner planet pair data
inputdata3 = "../data/input/Planet_and_Star_data_from_Q1_to_Q12_period_sorting_Pinner_2_sorted.txt" 
fx3 = open(inputdata2,'r')
datax3 = fx3.read()    
# Get column names
firstlinex3 = datax3.split('\n', 1)[0].split(' ')
firstlinex3 = [w.replace('"', '') for w in firstlinex3]
firstlinex3 = [w.replace('.', '_') for w in firstlinex3]
#print firstlinex1
fx3.close()
# organize data from each file name in list
datax3 = np.genfromtxt(inputdata3, dtype=None, skip_header=3,names=firstlinex3 )

#print(datax3['koi_slogg'])
#print(datax2['kepoi_name'])
#print(datax1['Re_V_ext'])

In [ ]:

mass_radius_powerlaw_model = """

data {    

    int<lower=1> Nm;
    int<lower=1> Ndata;
    real<lower=-1,upper=1> hhat[Ndata];
    real<lower=-1,upper=1> khat[Ndata];
    real<lower=0,upper=1> hhat_sigma[Ndata];
    real<lower=0,upper=1> khat_sigma[Ndata];
    
}

parameters {
    simplex[Nm] f;
    real<lower=0> e_sigma[Nm];
    real<lower=-1,upper=1> h[Ndata];
    real<lower=-1,upper=1> k[Ndata];

model {
        
    real ps_h[Nm];
    real ps_k[Nm];

    mass.const ~ uniform(0.0,3.0);
    mass.radius.exp ~ uniform(0.0,2.0);
    sigma.mass.radius ~ uniform(0.0,3.0); 
    
    // eccentricity distribution model section:
    e_sigma ~ uniform(0, 0.5);
    for (n in 1:Ndata)
      for (j in 1:Nm) {
        ps_h[j] <- log(f[j]) + normal_log(h[n],0.0,e_sigma[j]);
      }
      increment_log_prob(log_sum_exp(ps_h));
    for (n in 1:Ndata)
      for (j in 1:Nm) {
        ps_k[j] <- log(f[j]) + normal_log(k[n],0.0,e_sigma[j]);
      }
      increment_log_prob(log_sum_exp(ps_k));
    for (n in 1:Ndata)
      hhat[n] ~ normal(h[n], hhat_sigma[n]);
    for (n in 1:Ndata)
      khat[n] ~ normal(k[n], khat_sigma[n]);


    m_sun <- 332996.4274; // Msun in Mearth units
    r_sun <- 110.0;  // Rsun in Rearth units
    // m_star_const <- 1.37*m_sun;
    
    min_radius_s <- 0.5*r_sun; // put priors over this
    max_radius_s <- 2.0*r_sun; // put priors over this
    
    min_radius_p <- 0.1;
    max_radius_p <- 10.0;
    // radius_p_exp ~ uniform(0.0,2.0);
    
    min_mass_s <- 0.8*m.sun; // put priors over this
    max_mass_s <- 3.0*m.sun;  // put priors over this
    
    // Constants (depends on P.ext, P.int, j)
    pi <- 3.14;
    // radius_s_true <- r_sun;
    // mass_s_true <- m_sun;
    
    for (i in 1:Ndata) {
    
        // DEFINITIONS //
    
        C1[i] <- P_int[i] / ( ( pi * ( j[i]^(2/3) ) * ( (j[i]-1.0)^(1/3) * delta[i] ) ) );

        C2[i] <- 3.0 / ( 2.0 * delta[i] );
        
        C3[i] <- P_ext[i] / ( pi * j[i] * delta[i] );
    

        radius_ratio_true_int[i] <- radius_p_true_int[i] / radius_s_true[i];

        mass_ref_int[i] <- mass_const * ( radius_p_true_int[i]^mass_radius_exp );


        radius_ratio_true_ext[i] <- radius_p_true_ext[i] / radius_s_true[i];

        mass_ref_ext[i] <- mass_const * ( radius_p_true_ext[i]^mass_radius_exp );


        mass_ratio_true_int[i] <- mass_p_true_int[i] / mass_s_true[i];

        va_true_int[i] <- ( (-1.0) * C1[i] * mass_ratio_true_ext[i] * fc[i] ) - ( ( C1[i] * C2[i] * mass_ratio_true_ext[i] ) * ( ( fc[i] * h_true_int[i] ) + ( g[i] * h_true_ext[i] ) ) );

        vb_true_int[i] <- ( C1[i] * C2[i] * mass_ratio_true_ext[i] ) * ( ( fc[i] * k_true_int[i] ) + ( g[i] * k_true_ext[i] ) );


        mass_ratio_true_ext[i] <- mass_p_true_ext[i] / mass_s_true[i];

        va_true_ext[i] <- ( (-1.0) * C3[i] * mass_ratio_true_int[i] * g[i] ) + ( ( C2[i] * C3[i] * mass_ratio_true_int[i] ) * ( ( fc[i] * h_true_int[i] ) + ( g[i] * h_true_ext[i] ) ) );

        vb_true_ext[i] <- ( (-1.0) * C2[i] * C3[i] * mass_ratio_true_int[i] ) * ( ( fc[i] * k_true_int[i] ) + ( g[i] * k_true_ext[i] ) );
    

        // DISTRIBUTIONS //
	
	    radius_s_true[i] ~ uniform( min_radius_s, max_radius_s );
        
        radius_s_obs[i] ~ normal( radius_s_true[i], 1.0 / ( sigma_radius_s[i] * sigma_radius_s[i] ) ) %_% T(0,)
        mass.s.true[i] ~ uniform(min.mass.s,max.mass.s)
        mass.s.obs[i] ~ normal(mass.s.true[i],1.0/(sigma.mass.s[i]*sigma.mass.s[i]))

        radius.p.true.int[i] ~ uniform(min.radius.p, max.radius.p) 
        radius.ratio.obs.int[i] ~ normal(radius.ratio.true.int[i], 1.0/(sigma.radius.ratio.obs.int[i]*sigma.radius.ratio.obs.int[i]))

        mass.p.true.int[i] ~ normal(mass.ref.int[i],1.0/(sigma.mass.radius*sigma.mass.radius)) %_% T(0,)

        va.obs.int[i] ~ normal(va.true.int[i],1.0/(sigma.va.obs.int[i]*sigma.va.obs.int[i]))
        vb.obs.int[i] ~ normal(vb.true.int[i],1.0/(sigma.vb.obs.int[i]*sigma.vb.obs.int[i]))   

        radius.p.true.ext[i] ~ uniform(min.radius.p, max.radius.p) 
        radius.ratio.obs.ext[i] ~ normal(radius.ratio.true.ext[i], 1.0/(sigma.radius.ratio.obs.ext[i]*sigma.radius.ratio.obs.ext[i]))

        mass.p.true.ext[i] ~ normal(mass.ref.ext[i],1.0/(sigma.mass.radius*sigma.mass.radius)) %_% T(0,)

        va.obs.ext[i] ~ normal(va.true.ext[i],1.0/(sigma.va.obs.ext[i]*sigma.va.obs.ext[i]))
        vb.obs.ext[i] ~ normal(vb.true.ext[i],1.0/(sigma.vb.obs.ext[i]*sigma.vb.obs.ext[i]))
    }
}

"""

data = {}

#sm = StanModel(model_code=mass_radius_powerlaw_model)
#fit = sm.sampling(data=data, iter=10, chains=2, init=init, n_jobs=-1)

fit = pystan.stan(model_code=mass_radius_powerlaw_model, data=data, iter=1, chains=1, n_jobs=-1);

la = fit.extract(permuted=True)  # return a dictionary of arrays
alpha = la['mass.radius.exp']
beta = la['mass.const']
lnf0 = la['sigma.mass.radius']
print(mass.radius.exp)
print(mass.const)
print(sigma.mass.radius)

a = fit.extract(permuted=False)
print(a)
print(fit)

print(mass.radius.exp)

plt.hist(mass.radius.exp)
fit.plot()


